In [111]:
import numpy as np
import pandas as pd

A 0.5 MB file is going to be used to ilustrate how to reduce the memory consumption of the pandas dataframes. The file contains all the stops times where public transport operates inside Helsinki city. Using some pandas tips we will reduce up to  87 % of the initial memory overload of the standard pandas object.

In [112]:
df = pd.read_csv('C:/Users/Marc/Desktop/MUV/Datasets/Helsinki/gtfs/stop_times.txt')
print(df.head())

                   trip_id arrival_time departure_time  stop_id  \
0  1001_20180621_Ke_1_0535     05:35:00       05:35:00  1050417   
1  1001_20180621_Ke_1_0535     05:36:00       05:36:00  1050416   
2  1001_20180621_Ke_1_0535     05:38:00       05:38:00  1050408   
3  1001_20180621_Ke_1_0535     05:39:00       05:39:00  1050413   
4  1001_20180621_Ke_1_0535     05:40:00       05:40:00  1040401   

   stop_sequence           stop_headsign  pickup_type  drop_off_type  \
0              1  Käpylä via Lasipalatsi            0              1   
1              2  Käpylä via Lasipalatsi            0              0   
2              3  Käpylä via Lasipalatsi            0              0   
3              4  Käpylä via Lasipalatsi            0              0   
4              5  Käpylä via Lasipalatsi            0              0   

   shape_dist_traveled  timepoint  
0                0.000          1  
1                0.281          0  
2                0.959          0  
3                1.1

In [113]:
mem = df.memory_usage(deep=True).sum()/(1024*1024)
print('memory on MB:')
print(mem)
print('\nData types\n')
print(df.dtypes)
print('\nQuantity of nuls\n')
print(df.isnull().sum())

memory on MB:
1684.1118535995483

Data types

trip_id                 object
arrival_time            object
departure_time          object
stop_id                  int64
stop_sequence            int64
stop_headsign           object
pickup_type              int64
drop_off_type            int64
shape_dist_traveled    float64
timepoint                int64
dtype: object

Quantity of nuls

trip_id                     0
arrival_time                0
departure_time              0
stop_id                     0
stop_sequence               0
stop_headsign          245690
pickup_type                 0
drop_off_type               0
shape_dist_traveled         0
timepoint                   0
dtype: int64


Now we know the dataframe consumes 1684.11 MB of memory. The most consuming data type is object type, so in case that can't be converted to float or int, the category type can be considered. For the float type we can check if it's possible to convert it to int depending on the existence of NaNs and the floating values. If can't be converted to int, downcast will be useful to use less bites for value. The int type is the most efficient, downcast will be good to decrease the memory overload.

Abstract:
condtion to be int: be numeric, not floating values, not NaN values
condition to be float: floating values, has NaN 
condition to be category: more than a half of the records are not unique (floats or objects)

The following code willl compute the distinct values for every column

In [114]:
for col in df.columns:
    print(col)
    print(df[col].value_counts())

trip_id
7667_20180621_Ma_2_0715     87
9788K_20180621_To_1_1552    87
9788K_20180621_Ma_1_1215    87
9788K_20180621_Ti_1_0755    87
9788K_20180621_Ke_1_0900    87
9788K_20180621_Ke_1_1730    87
9788K_20180621_Ti_1_0900    87
9788K_20180621_To_1_0755    87
9788K_20180621_Ke_1_1552    87
9788K_20180621_Su_1_1440    87
9788K_20180621_Pe_2_0750    87
9788K_20180621_La_2_1215    87
7667_20180621_Ti_2_0715     87
9788K_20180621_Ma_2_0605    87
9788K_20180621_Ma_2_0750    87
9788K_20180621_To_1_1730    87
9788K_20180621_JP_1_1440    87
7667_20180621_To_2_0715     87
9788K_20180621_Ma_1_1552    87
9788K_20180621_Ma_1_0755    87
9788K_20180621_Ti_2_0605    87
9788K_20180621_To_2_1230    87
9788K_20180621_JP_2_1615    87
9788K_20180621_Pe_1_1215    87
9788K_20180621_Su_1_1840    87
7667_20180621_Pe_2_0715     87
9788K_20180621_Su_2_1615    87
9788K_20180621_La_1_1840    87
9788K_20180621_Ma_2_1230    87
9788K_20180621_Pe_1_0755    87
                            ..
1019_20180621_To_1_0850      2


0.000     189639
1.862       5534
10.519      4347
6.134       4301
2.967       4145
1.174       4099
2.975       4006
2.259       3999
2.603       3996
1.035       3897
3.326       3889
0.820       3876
16.824      3843
0.332       3771
1.647       3771
6.976       3758
13.706      3758
6.061       3743
2.880       3686
5.458       3617
0.736       3589
7.823       3451
3.896       3420
3.479       3226
4.162       3128
1.951       3115
12.133      3092
7.658       3081
5.057       3037
7.190       3014
           ...  
24.545         1
15.035         1
19.633         1
24.923         1
28.520         1
17.792         1
27.169         1
21.032         1
40.208         1
32.713         1
43.920         1
16.921         1
29.776         1
9.343          1
22.340         1
18.234         1
1.452          1
28.128         1
31.495         1
38.036         1
39.119         1
26.682         1
47.286         1
0.872          1
37.562         1
11.697         1
26.374         1
18.689        

In [123]:
df_new = pd.DataFrame(index = df.index)

def memory_saving(df, df_new, columns):
    memory = df[columns].memory_usage(deep=True).sum()/(1024*1024)
    memory_new = df_new[columns].memory_usage(deep=True).sum()/(1024*1024)  
    print('memory save (MB): ' + str(memory - memory_new))


def DownCastInt(df, df_new):
    # takes a numeric column and returns it's lowest int type. 
    int_cols = df.select_dtypes(include=['integer']).columns
    for col in int_cols:
        df_new[col] = pd.to_numeric(df[col], downcast='integer')
        print(col + ': ' + str(df[col].dtype) + ' ==> ' + str(df_new[col].dtype))

    memory_saving(df, df_new, int_cols)
    return df_new


def DownCastFloat(df, new_df):
    float_cols = df.select_dtypes(include=['float']).columns
    for col in float_cols:
        df_new[col] = pd.to_numeric(df[col], downcast='float')
        print(col + ': ' + str(df[col].dtype) + ' ==> ' + str(df_new[col].dtype))
    memory_saving(df, df_new, float_cols)
    return df_new


def to_categorical(df, df_new):
    obj_cols = df.select_dtypes(include=['object']).columns
    
    for col in obj_cols:        
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df_new[col] = df[col].astype('category')    
            print(col + ': ' + str(df[col].dtype) + ' ==> ' + str(df_new[col].dtype))
    memory_saving(df, df_new, obj_cols)
    
    return df_new

In [124]:
df_new = DownCastInt(df, df_new)

stop_id: int64 ==> int32
stop_sequence: int64 ==> int8
pickup_type: int64 ==> int8
drop_off_type: int64 ==> int8
timepoint: int64 ==> int8
memory save (MB): 157.8863525390625


In [125]:
df_new = to_categorical(df, df_new)

trip_id: object ==> category
arrival_time: object ==> category
departure_time: object ==> category
stop_headsign: object ==> category
memory save (MB): 1378.0406789779663


In [126]:
# We can multiply records in order not have any value after the comma. If we don't have any NaN, we are able to trasform into INT.

df_new['shape_dist_traveled(m)']  = (df['shape_dist_traveled']*1000).astype(int)

memory = df['shape_dist_traveled'].memory_usage(deep=True)/(1024*1024)
memory_new = df_new['shape_dist_traveled(m)'].memory_usage(deep=True)/(1024*1024)

print('memory save (MB): ' + str(memory - memory_new))

memory save (MB): 19.735794067382812


In [130]:
print(df_new.dtypes)
print()
print('original memory: ', df.memory_usage(deep=True).sum()/(1024*1024))
print('new memory: ', df_new.memory_usage(deep=True).sum()/(1024*1024))

stop_id                      int32
stop_sequence                 int8
pickup_type                   int8
drop_off_type                 int8
timepoint                     int8
trip_id                   category
arrival_time              category
departure_time            category
stop_headsign             category
shape_dist_traveled(m)       int32
dtype: object

original memory:  1684.1118535995483
new memory:  128.44902801513672


Using this proces we are able to save 1156 MB of memory and work with better huge files in terms of efficiency, speed and memory overload.